In [ ]:
print('ACIT 4610, Portfolio-2024')
print('\nGroup-13')
print('\nparticipate: Thomas, M khan, Harith')

In [2]:
print('Tools and Libraries:')
print('\n1- Python 3.')
print('\n2- OpenAI Gym for the environment.')
print('\n3- Deep to use Genetic Programming, implement an evolutionary algorithms.')
print('\n4- NumPy for numerical computations.')
print('\n5- pandas for data manipulation and analysis.')
print('\n6- requests for the process of sending and receiving data from websites.')
print('\n7- Matplotlib for plotting results.')
print('\n8- TensorFlow/PyTorch for more advanced RL algorithms like DQN.')
print('\n9- pymoo for for multi-objective optimization algorithms.')
print('\n10-Keras for the convenience of mathematical calculations')

!pip3 freeze
!pip3 install gym numpy tensorflow keras openAi matplotlib requests pandas gym deap pymoo

import sys
print('System version',sys.version)

Tools and Libraries:

1- Python 3.

2- OpenAI Gym for the environment.

3- Deep to use Genetic Programming, implement an evolutionary algorithms.

4- NumPy for numerical computations.

5- pandas for data manipulation and analysis.

6- requests for the process of sending and receiving data from websites.

7- Matplotlib for plotting results.

8- TensorFlow/PyTorch for more advanced RL algorithms like DQN.

9- pymoo for for multi-objective optimization algorithms.

10-Keras for the convenience of mathematical calculations
about-time==4.2.1
alive-progress==3.1.5
asttokens==2.4.1
autograd==1.7.0
beautifulsoup4==4.12.3
certifi==2024.8.30
charset-normalizer==3.4.0
cloudpickle==3.1.0
cma==3.2.2
colorama==0.4.6
comm==0.2.2
contourpy==1.3.0
cycler==0.12.1
deap==1.4.1
debugpy==1.8.7
decorator==5.1.1
Deprecated==1.2.14
dill==0.3.9
executing==2.1.0
Farama-Notifications==0.0.4
fonttools==4.54.1
frozendict==2.4.6
grapheme==0.6.0
gymnasium==1.0.0
html5lib==1.1
idna==3.10
ipykernel==6.29.5
ipython==8.2

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow


Problem 3: Solving the Vehicle Routing Problem with Time Windows (VRPTW) Using 
Ant Colony Optimization (ACO) and Particle Swarm Optimization (PSO)

Overview 
This exercise focuses on optimizing the delivery routes for a fleet of vehicles using two nature
inspired optimization algorithms: Ant Colony Optimization (ACO) and Particle Swarm 
Optimization (PSO). The objective is to find the most efficient routes for a set of vehicles, 
ensuring that all customers receive their deliveries within specified time windows. You will 
implement both ACO and PSO to solve the Vehicle Routing Problem with Time Windows 
(VRPTW) and compare the effectiveness of these algorithms. 

Problem Description 
The VRPTW is a variant of the Vehicle Routing Problem (VRP), which is crucial in logistics, 
transportation, and supply chain management. In VRPTW, a fleet of vehicles must deliver 
goods to multiple customers. Each customer has a specific time window during which the 
delivery must occur. The challenge is to design routes that minimize the total travel distance 
while ensuring all deliveries meet their respective time constraints.

The primary objective of the VRPTW is to minimize the total distance travelled by all vehicles 
while ensuring that: 
1. Each customer is visited exactly once by one vehicle. 
2. Deliveries occur within the specified time windows. 
3. The total demand on any route does not exceed the vehicle's capacity. 

Key Components 
• Depot: The starting and ending point for all vehicles. 
• Customers: Each customer has a location, a demand for goods, a time window for 
receiving the delivery, and a service time. 
• Vehicles: Each vehicle has a limited capacity and must return to the depot after 
completing its route. 

In [4]:
# 1. Dataset, and Preprocessing
#https://www.sintef.no/projectweb/top/vrptw/100-customers/

# Read Text Files with Pandas using read_table()

import re

# Path to the file containing the dataset
file_path = 'c101.txt'

# Function to read the file and parse the routes
def parse_routes_from_file(file_path):
    with open(file_path, 'r') as file:
        # Read the content of the file
        dataset = file.read()

    # Regular expression pattern to match the routes
    route_pattern = re.compile(r"Route\s+(\d+)\s*:\s*([0-9\s]+)")

    # Find all routes using the regex pattern
    routes = {}

    matches = route_pattern.findall(dataset)

    # Process the matches to store routes as integers in a dictionary
    for match in matches:
        route_number = int(match[0])
        route_values = list(map(int, match[1].split()))
        routes[route_number] = route_values

    return routes

# Call the function and get the routes
routes = parse_routes_from_file(file_path)

# Print the parsed routes
for route_number, route in routes.items():
    print(f"Route {route_number}: {route}")



Route 1: [81, 78, 76, 71, 70, 73, 77, 79, 80]
Route 2: [57, 55, 54, 53, 56, 58, 60, 59]
Route 3: [98, 96, 95, 94, 92, 93, 97, 100, 99]
Route 4: [32, 33, 31, 35, 37, 38, 39, 36, 34]
Route 5: [13, 17, 18, 19, 15, 16, 14, 12]
Route 6: [90, 87, 86, 83, 82, 84, 85, 88, 89, 91]
Route 7: [43, 42, 41, 40, 44, 46, 45, 48, 51, 50, 52, 49, 47]
Route 8: [67, 65, 63, 62, 74, 72, 61, 64, 68, 66, 69]
Route 9: [5, 3, 7, 8, 10, 11, 9, 6, 4, 2, 1, 75]
Route 10: [20, 24, 25, 27, 29, 30, 28, 26, 23, 22, 21]


1. Implement Ant Colony Optimization (ACO) for VRPTW: 
• Develop an ACO algorithm tailored to the VRPTW. 
• Optimize the routes to minimize total travel distance while respecting the time 
windows. 
• Compare the results of ACO with PSO. 

In [1]:
import numpy as np
import random

class ACO_VRPTW:
    def __init__(self, distance_matrix, time_windows, demand, num_vehicles, vehicle_capacity, alpha=1.0, beta=2.0, rho=0.1, q0=0.9, iterations=100, ants=10):
        # Problem Parameters
        self.distance_matrix = distance_matrix
        self.time_windows = time_windows
        self.demand = demand
        self.num_customers = len(distance_matrix)
        self.num_vehicles = num_vehicles
        self.vehicle_capacity = vehicle_capacity

        # ACO Parameters
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q0 = q0
        self.iterations = iterations
        self.ants = ants

        # Initialize pheromones
        self.pheromone = np.ones((self.num_customers, self.num_customers))

        # Solution matrices
        self.best_route = None
        self.best_cost = float('inf')

    def calculate_cost(self, route):
        total_distance = 0
        total_time = 0
        current_time = 0  # Track the time at which each customer is served
        
        # Route costs and time windows check
        for vehicle in route:
            last_customer = 0  # Start at the depot
            current_time = 0
            for customer in vehicle:
                dist = self.distance_matrix[last_customer][customer]
                total_distance += dist
                travel_time = dist / 30  # assuming average speed of 30 units per time
                current_time += travel_time
                
                # Check if customer can be serviced within time window
                if current_time < self.time_windows[customer][0]:
                    current_time = self.time_windows[customer][0]  # Wait until the time window opens
                if current_time > self.time_windows[customer][1]:
                    return float('inf')  # Infeasible solution
                
                last_customer = customer
                
        return total_distance

    def choose_next_customer(self, current_customer, visited, vehicle_capacity, current_time):
        # Compute probabilities of choosing next customer based on pheromones and heuristic information
        probabilities = []
        for next_customer in range(self.num_customers):
            if next_customer not in visited:
                dist = self.distance_matrix[current_customer][next_customer]
                if self.demand[next_customer] <= vehicle_capacity:
                    heuristic = 1 / (dist + 1e-6)  # Inverse of distance as heuristic
                    pheromone = self.pheromone[current_customer][next_customer]
                    probability = (pheromone ** self.alpha) * (heuristic ** self.beta)
                    probabilities.append(probability)
                else:
                    probabilities.append(0)
            else:
                probabilities.append(0)

        total_prob = sum(probabilities)
        if total_prob == 0:
            return -1  # No valid next customer
        probabilities = [p / total_prob for p in probabilities]

        # Select the next customer based on probability
        next_customer = np.random.choice(range(self.num_customers), p=probabilities)
        return next_customer

    def construct_solution(self):
        routes = [[] for _ in range(self.num_vehicles)]
        visited = [False] * self.num_customers
        vehicle_capacity = [self.vehicle_capacity] * self.num_vehicles

        for vehicle_idx in range(self.num_vehicles):
            current_customer = 0  # Start at depot
            current_time = 0
            route = routes[vehicle_idx]
            while len(route) < self.num_customers:
                next_customer = self.choose_next_customer(current_customer, visited, vehicle_capacity[vehicle_idx], current_time)
                if next_customer == -1:
                    break

                visited[next_customer] = True
                route.append(next_customer)
                current_customer = next_customer
                vehicle_capacity[vehicle_idx] -= self.demand[next_customer]
                current_time += self.distance_matrix[current_customer][next_customer] / 30  # assuming speed of 30 units

        return routes

    def update_pheromones(self, routes):
        delta_pheromone = np.zeros_like(self.pheromone)

        # Update pheromones based on routes
        for route in routes:
            route_cost = self.calculate_cost([route])
            if route_cost == float('inf'):
                continue
            pheromone_deposit = 1.0 / route_cost
            for vehicle in route:
                for i in range(len(vehicle) - 1):
                    current = vehicle[i]
                    next_customer = vehicle
